<a href="https://colab.research.google.com/github/adzetto/DASK_26__DESIGN_AND_ANALYSIS/blob/main/notebooks/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DASK 2026 Twin Towers - Final Structural Analysis

## Advanced Modal Analysis with OpenSees & TBDY 2018 Spectrum

**Author:** Muhammet Yağcıoğlu  
**Date:** February 2026  
**Model:** V8 Optimized Balsa Wood Structure

---

### Executive Summary

This notebook presents a complete eigenvalue analysis of the DASK Twin Towers model using the **OpenSees** finite element framework.

## 1. Theoretical Background

### 1.1 Equation of Motion

The undamped free vibration equation for a multi-degree-of-freedom (MDOF) system is:

$$\mathbf{M}\ddot{\mathbf{u}} + \mathbf{K}\mathbf{u} = \mathbf{0}$$

where:
- $\mathbf{M}$ = Mass matrix $(n \times n)$
- $\mathbf{K}$ = Stiffness matrix $(n \times n)$
- $\mathbf{u}$ = Displacement vector $(n \times 1)$

### 1.2 Eigenvalue Problem

Assuming harmonic motion $\mathbf{u}(t) = \boldsymbol{\phi} \sin(\omega t)$, we obtain the generalized eigenvalue problem:

$$\mathbf{K}\boldsymbol{\phi} = \omega^2 \mathbf{M}\boldsymbol{\phi}$$

or equivalently:

$$(\mathbf{K} - \omega^2 \mathbf{M})\boldsymbol{\phi} = \mathbf{0}$$

The non-trivial solution requires:

$$\det(\mathbf{K} - \omega^2 \mathbf{M}) = 0$$

### 1.3 Modal Parameters

From the eigenvalues $\lambda_i = \omega_i^2$:

| Parameter | Formula | Unit |
|-----------|---------|------|
| Angular Frequency | $\omega_i = \sqrt{\lambda_i}$ | rad/s |
| Natural Period | $T_i = \dfrac{2\pi}{\omega_i}$ | s |
| Natural Frequency | $f_i = \dfrac{1}{T_i}$ | Hz |

## 2. Stiffness Matrix Formulation

### 2.1 Element Stiffness (Euler-Bernoulli Beam)

For a 3D beam element with 6 DOF per node, the local stiffness matrix is:

$$\mathbf{k}_e = \begin{bmatrix}
\dfrac{EA}{L} & 0 & 0 & 0 & 0 & 0 & -\dfrac{EA}{L} & 0 & 0 & 0 & 0 & 0 \\
0 & \dfrac{12EI_z}{L^3} & 0 & 0 & 0 & \dfrac{6EI_z}{L^2} & 0 & -\dfrac{12EI_z}{L^3} & 0 & 0 & 0 & \dfrac{6EI_z}{L^2} \\
0 & 0 & \dfrac{12EI_y}{L^3} & 0 & -\dfrac{6EI_y}{L^2} & 0 & 0 & 0 & -\dfrac{12EI_y}{L^3} & 0 & -\dfrac{6EI_y}{L^2} & 0 \\
0 & 0 & 0 & \dfrac{GJ}{L} & 0 & 0 & 0 & 0 & 0 & -\dfrac{GJ}{L} & 0 & 0 \\
\vdots & & & & \ddots & & & & & & & \vdots \\
\end{bmatrix}$$

### 2.2 Assembly Process

$$\mathbf{K} = \sum_{e=1}^{n_{elem}} \mathbf{T}_e^T \mathbf{k}_e \mathbf{T}_e$$

where $\mathbf{T}_e$ is the transformation matrix from local to global coordinates.

## 3. Model Data Import (V8 Matrices)

### 3.1 Position Matrix $\mathbf{X}$

The position matrix contains nodal coordinates:

$$\mathbf{X} = \begin{bmatrix}
x_1 & y_1 & z_1 \\
x_2 & y_2 & z_2 \\
\vdots & \vdots & \vdots \\
x_n & y_n & z_n
\end{bmatrix} \in \mathbb{R}^{n \times 3}$$

In [1]:
# ===========
# IMPORTS
# ===========
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scienceplots
from matplotlib.patches import FancyBboxPatch
import openseespy.opensees as ops
from pathlib import Path

# Academic plotting style
plt.style.use(['science', 'ieee'])

np.set_printoptions(precision=4, suppress=True)
pd.set_option('display.max_columns', 20)
pd.set_option('display.float_format', '{:.4f}'.format)

DATA_DIR = Path('/content/DASK_26__DESIGN_AND_ANALYSIS/data')
RESULTS_DIR = Path('/content/DASK_26__DESIGN_AND_ANALYSIS/results/data')
VIZ_DIR = Path('/content/DASK_26__DESIGN_AND_ANALYSIS/results/visualizations')

In [2]:
# ============
# 3.2 LOAD V8
# ============
# Position Matrix X (Nodal Coordinates)
pos_df = pd.read_csv(DATA_DIR / 'twin_position_matrix.csv')

# (Element Topology)
conn_df = pd.read_csv(DATA_DIR / 'twin_connectivity_matrix.csv')

# Extract coordinate matrix
X = pos_df[['x', 'y', 'z']].values

print("POSITION MATRIX X")
print(f"Shape: {X.shape} (nodes × coordinates)")
print(f"\nDimensions:")
print(f"  X range: [{X[:, 0].min():.1f}, {X[:, 0].max():.1f}] cm")
print(f"  Y range: [{X[:, 1].min():.1f}, {X[:, 1].max():.1f}] cm")
print(f"  Z range: [{X[:, 2].min():.1f}, {X[:, 2].max():.1f}] cm")
print(f"\nFirst 10 nodes:")
pos_df.head(10)

POSITION MATRIX X
Shape: (1680, 3) (nodes × coordinates)

Dimensions:
  X range: [0.0, 30.0] cm
  Y range: [0.0, 40.0] cm
  Z range: [0.0, 153.0] cm

First 10 nodes:


,node_id,x,y,z,floor,zone,tower
0,0,0.0000,0.0000,0.0000,0,tower,1
1,1,0.0000,7.4000,0.0000,0,tower,1
2,2,0.0000,8.6000,0.0000,0,tower,1
3,3,0.0000,16.0000,0.0000,0,tower,1
4,4,3.0000,0.0000,0.0000,0,tower,1
5,5,3.0000,7.4000,0.0000,0,tower,1
6,6,3.0000,8.6000,0.0000,0,tower,1
7,7,3.0000,16.0000,0.0000,0,tower,1
8,8,11.0000,0.0000,0.0000,0,tower,1
9,9,11.0000,7.4000,0.0000,0,tower,1


In [3]:
# ==============================================================================
# 3.3 CONNECTIVITY MATRIX C
# ==============================================================================

# Extract connectivity as numpy array [element_id, node_i, node_j]
C = conn_df[['element_id', 'node_i', 'node_j']].values

# Element type distribution
elem_stats = conn_df.groupby('element_type').agg(
    count=('element_id', 'count'),
    avg_length=('length', 'mean')
).round(2)

print("=" * 60)
print("CONNECTIVITY MATRIX C")
print("=" * 60)
print(f"Shape: {C.shape} (elements × [id, node_i, node_j])")
print(f"\nElement Statistics:")
elem_stats

CONNECTIVITY MATRIX C
Shape: (4232, 3) (elements × [id, node_i, node_j])

Element Statistics:


,count,avg_length
element_type,,
beam_x,1144,4.1500
beam_y,936,5.3300
brace_xz,208,7.5300
brace_yz,64,10.0600
bridge_beam,40,4.8000
bridge_column,8,7.5000
bridge_rigid,16,11.3100
bridge_truss,32,8.5700
column,1600,6.1200


## 4. Material Properties

### 4.1 Balsa Wood Properties

| Property | Symbol | Value | Unit |
|----------|--------|-------|------|
| Young's Modulus | $E$ | 3.7 | GPa |
| Shear Modulus | $G = E/2.6$ | 1.42 | GPa |
| Density | $\rho$ | 160 | kg/m³ |
| Poisson's Ratio | $\nu$ | 0.30 | - |

### 4.2 Section Properties

For a square cross-section with side $b$:

$$A = b^2, \quad I = \frac{b^4}{12}, \quad J \approx 0.141 b^4$$

In [4]:
# ==============================================================================
# 4.3 MATERIAL AND SECTION PARAMETERS
# ==============================================================================

# Material Properties (SI Units: kN, m, tonne, s)
PARAMS = pd.DataFrame({
    'Parameter': ['E_balsa', 'G_balsa', 'rho_balsa', 'b_section', 'A', 'I', 'J'],
    'Value': [3.7e6, 1.42e6, 0.16, 0.003, 9e-6, 6.75e-12, 1.14e-11],
    'Unit': ['kN/m²', 'kN/m²', 't/m³', 'm', 'm²', 'm⁴', 'm⁴'],
    'Description': [
        'Young\'s Modulus (Balsa)',
        'Shear Modulus',
        'Density',
        'Section side (3mm)',
        'Cross-sectional area',
        'Second moment of area',
        'Torsional constant'
    ]
})

PARAMS.style.set_properties(**{'text-align': 'left'}).hide(axis='index')

Parameter,Value,Unit,Description
E_balsa,3700000.000000,kN/m²,Young's Modulus (Balsa)
G_balsa,1420000.000000,kN/m²,Shear Modulus
rho_balsa,0.160000,t/m³,Density
b_section,0.003000,m,Section side (3mm)
A,0.000009,m²,Cross-sectional area
I,0.000000,m⁴,Second moment of area
J,0.000000,m⁴,Torsional constant


## 5. OpenSees Model Construction

### 5.1 Model Domain

The OpenSees model is constructed in a 3D space with 6 degrees of freedom per node:

$$\text{DOF} = \{u_x, u_y, u_z, \theta_x, \theta_y, \theta_z\}$$

### 5.2 Coordinate System

- **X-axis**: Longitudinal (along bridge)
- **Y-axis**: Transverse
- **Z-axis**: Vertical (height)

In [5]:
# =========================
# 5.3 BUILD OPENSEES MODEL
# =========================

# Initialize
ops.wipe()
ops.model('basic', '-ndm', 3, '-ndf', 6)

# Material parameters
E = PARAMS.loc[PARAMS['Parameter'] == 'E_balsa', 'Value'].iloc[0]
G = PARAMS.loc[PARAMS['Parameter'] == 'G_balsa', 'Value'].iloc[0]
rho = PARAMS.loc[PARAMS['Parameter'] == 'rho_balsa', 'Value'].iloc[0]
A = PARAMS.loc[PARAMS['Parameter'] == 'A', 'Value'].iloc[0]
I = PARAMS.loc[PARAMS['Parameter'] == 'I', 'Value'].iloc[0]
J = PARAMS.loc[PARAMS['Parameter'] == 'J', 'Value'].iloc[0]

# Scale factor: cm to m
SCALE = 1

In [6]:
# =================
# 5.4 CREATE NODES
# =================

# Apply node creation using pandas apply (no explicit loop)
_ = pos_df.apply(
    lambda row: ops.node(
        int(row['node_id']),
        row['x'] * SCALE,
        row['y'] * SCALE,
        row['z'] * SCALE
    ),
    axis=1
)

# Fix base nodes (floor == 0). Use boolean indexing.
# Should be run one time after all nodes are created, to avoid redundant calls.
base_nodes = pos_df[pos_df['floor'] == 0]['node_id'].astype(int).tolist()
_ = [ops.fix(nid, 1, 1, 1, 1, 1, 1) for nid in base_nodes]

print(f"Nodes created: {len(pos_df)}")
print(f"Base nodes fixed: {len(base_nodes)}")

Nodes created: 1680
Base nodes fixed: 64


In [7]:
# ==============================================================================
# 5.5 DEFINE TRANSFORMATIONS AND MATERIALS
# ==============================================================================

# Geometric transformations (local to global coordinate mapping)
# transf_id, vec_xz (orientation vector)
transformations = pd.DataFrame({
    'id': [1, 2, 3],
    'vec': [[0, 1, 0], [1, 0, 0], [0, 1, 0]],
    'use': ['horizontal_x', 'horizontal_y', 'vertical']
})

# Apply transformations
_ = transformations.apply(
    lambda r: ops.geomTransf('Linear', int(r['id']), *r['vec']),
    axis=1
)

# Uniaxial material for truss elements
ops.uniaxialMaterial('Elastic', 1, E)

transformations

,id,vec,use
0,1,"[0, 1, 0]",horizontal_x
1,2,"[1, 0, 0]",horizontal_y
2,3,"[0, 1, 0]",vertical


In [8]:
# =====================
# 5.6 CREATE ELEMENTS
# =====================

node_coords = pos_df.set_index('node_id')[['x', 'y', 'z']].mul(SCALE).to_dict('index')

PIN_TYPES = {'brace_xz', 'brace_yz', 'floor_brace', 'bridge_truss',
             'shear_wall_xz', 'shear_wall_yz'}

conn_df['is_truss'] = conn_df['element_type'].isin(PIN_TYPES)

mass_density = A * rho  # tonne/m

def create_element(row):
    eid = int(row['element_id'])
    ni, nj = int(row['node_i']), int(row['node_j'])

    pi, pj = node_coords[ni], node_coords[nj]
    dx = abs(pi['x'] - pj['x'])
    dy = abs(pi['y'] - pj['y'])
    dz = abs(pi['z'] - pj['z'])

    if row['is_truss']:
        ops.element('Truss', eid, ni, nj, A, 1)
    else:
        transf = 3 if dz > max(dx, dy) * 0.1 else (1 if dx > dy else 2)
        ops.element('elasticBeamColumn', eid, ni, nj, A, E, G, J, I, I, transf, '-mass', mass_density)

_ = conn_df.apply(create_element, axis=1)

elem_summary = conn_df.groupby('is_truss').size().rename({True: 'Truss', False: 'Beam'})
print(f"Elements created: {len(conn_df)}")
print(f"  Truss: {elem_summary.get('Truss', 0)}")
print(f"  Beam:  {elem_summary.get('Beam', 0)}")

Elements created: 4232
  Truss: 488
  Beam:  3744


In [9]:
# ==========================
# 5.7 APPLY NODAL MASSES
# ==========================

# Calculate mass per floor level (lumped mass approach)
# Total model mass ~ 1.5 kg distributed across floors
TOTAL_MASS = 1.5e-3  # tonnes

# Floor node counts
floor_counts = pos_df.query('floor > 0').groupby('floor').size()
n_free_nodes = floor_counts.sum()
mass_per_node = TOTAL_MASS / n_free_nodes

# Apply masses to non-base nodes
free_nodes = pos_df.query('floor > 0')['node_id'].astype(int).tolist()
_ = [ops.mass(nid, mass_per_node, mass_per_node, mass_per_node, 0, 0, 0) for nid in free_nodes]

print(f"Total mass: {TOTAL_MASS * 1000:.2f} kg")
print(f"Mass per node: {mass_per_node * 1e6:.2f} g")
print(f"Nodes with mass: {len(free_nodes)}")

NumExprClobberingError: Variables in expression "(floor) > (0)" overlap with builtins: ('floor')

## 6. Eigenvalue Analysis

### 6.1 Solution Method

OpenSees uses the **Lanczos** algorithm to solve the generalized eigenvalue problem:

$$\mathbf{K}\boldsymbol{\phi}_i = \lambda_i \mathbf{M}\boldsymbol{\phi}_i$$

The algorithm efficiently computes the lowest $n$ eigenvalues for sparse systems.

In [10]:
# ==============================================================================
# 6.2 PERFORM EIGENVALUE ANALYSIS
# ==============================================================================

N_MODES = 8

eigenvalues = np.array(ops.eigen(N_MODES))

omega = np.sqrt(eigenvalues)           # Angular frequency (rad/s)
periods = 2 * np.pi / omega            # Natural period (s)
frequencies = 1 / periods              # Natural frequency (Hz)

modal_results = pd.DataFrame({
    'Mode': np.arange(1, N_MODES + 1),
    'Eigenvalue': eigenvalues,
    'Omega (rad/s)': omega,
    'Period (s)': periods,
    'Frequency (Hz)': frequencies
})


modal_results.style.format({
    'Eigenvalue': '{:.2e}',
    'Omega (rad/s)': '{:.2f}',
    'Period (s)': '{:.4f}',
    'Frequency (Hz)': '{:.2f}'
}).hide(axis='index')

Mode,Eigenvalue,Omega (rad/s),Period (s),Frequency (Hz)
1,1.27e-04,0.01,558.2600,0.00
2,4.95e-04,0.02,282.2917,0.00
3,5.57e-04,0.02,266.3204,0.00
4,1.13e-03,0.03,187.0361,0.01
5,2.19e-03,0.05,134.3753,0.01
6,3.12e-03,0.06,112.4821,0.01
7,3.92e-03,0.06,100.4055,0.01
8,4.77e-03,0.07,90.9747,0.01


## 7. TBDY 2018 Design Spectrum

### 7.1 Spectral Acceleration Formula

The horizontal elastic design spectrum $S_{ae}(T)$ is defined as:

$$S_{ae}(T) = \begin{cases}
(0.4 + 0.6 \frac{T}{T_A}) S_{DS} & 0 \leq T < T_A \\
S_{DS} & T_A \leq T < T_B \\
\frac{S_{D1}}{T} & T_B \leq T < T_L \\
\frac{S_{D1} T_L}{T^2} & T \geq T_L
\end{cases}$$

### 7.2 AFAD Parameters (Istanbul, ZD Soil)

| Level | Return Period | $S_{DS}$ (g) | $S_{D1}$ (g) | $T_A$ (s) | $T_B$ (s) |
|-------|---------------|--------------|--------------|-----------|-----------|
| DD-1  | 2475 yr       | 1.544        | 0.800        | 0.104     | 0.518     |
| DD-2  | 475 yr        | 1.008        | 0.514        | 0.102     | 0.510     |
| DD-3  | 72 yr         | 0.542        | 0.238        | 0.088     | 0.438     |
| DD-4  | 43 yr         | 0.384        | 0.158        | 0.083     | 0.412     |

**Source:** AFAD TDTH (tdth.afad.gov.tr), Coordinates: 41.002136°N, 29.106832°E

In [ ]:
# =====================================
# 7.3 TBDY 2018 SPECTRUM CALCULATION
# =====================================

SPEC = {
    'SDS': 2.046,  # g
    'SD1': 0.619,  # g
    'TA': 0.061,   # s
    'TB': 0.303,   # s
    'TL': 6.0      # s
}

T_range = np.linspace(0.001, 2.0, 10000)

Sae = np.select(
    [
        T_range < SPEC['TA'],                              # Ascending
        (T_range >= SPEC['TA']) & (T_range < SPEC['TB']),  # Plateau
        (T_range >= SPEC['TB']) & (T_range < SPEC['TL']),  # Descending
        T_range >= SPEC['TL']                              # Long period
    ],
    [
        (0.4 + 0.6 * T_range / SPEC['TA']) * SPEC['SDS'],  # Ascending formula
        np.full_like(T_range, SPEC['SDS']),                # Plateau (constant)
        SPEC['SD1'] / T_range,                             # Descending formula
        SPEC['SD1'] * SPEC['TL'] / T_range**2              # Long period formula
    ],
    default=0
)

spectrum_df = pd.DataFrame({'Period': T_range, 'Sae': Sae})

T_modes = modal_results['Period (s)'].values
Sae_modes = np.select(
    [
        T_modes < SPEC['TA'],
        (T_modes >= SPEC['TA']) & (T_modes < SPEC['TB']),
        T_modes >= SPEC['TB']
    ],
    [
        (0.4 + 0.6 * T_modes / SPEC['TA']) * SPEC['SDS'],
        np.full_like(T_modes, SPEC['SDS']),
        SPEC['SD1'] / T_modes
    ],
    default=0
)
modal_results['Sae (g)'] = Sae_modes

print(f"  SDS = {SPEC['SDS']:.3f} g")
print(f"  TA  = {SPEC['TA']:.3f} s")
print(f"  TB  = {SPEC['TB']:.3f} s")
print(f"  TL  = {SPEC['TL']:.1f} s")
print(Sae_modes)

In [ ]:
# ============================
# 7.4 SPECTRUM VISUALIZATION
# ============================

# AFAD TBDY 2018 Parameters for all DD levels (Istanbul, ZD soil)
DD_PARAMS = {
    'DD-1': {'SDS': 1.544, 'SD1': 0.800, 'TA': 0.104, 'TB': 0.518, 'TL': 6.0},
    'DD-2': {'SDS': 1.008, 'SD1': 0.514, 'TA': 0.102, 'TB': 0.510, 'TL': 6.0},
    'DD-3': {'SDS': 0.542, 'SD1': 0.238, 'TA': 0.088, 'TB': 0.438, 'TL': 6.0},
    'DD-4': {'SDS': 0.384, 'SD1': 0.158, 'TA': 0.083, 'TB': 0.412, 'TL': 6.0},
}

def compute_spectrum(T, SDS, SD1, TA, TB, TL):
    """Compute Sae(T) per TBDY 2018"""
    return np.select(
        [T < TA, (T >= TA) & (T < TB), (T >= TB) & (T < TL), T >= TL],
        [(0.4 + 0.6 * T / TA) * SDS, SDS, SD1 / T, SD1 * TL / T**2],
        default=0
    )

T_range = np.linspace(0.001, 2.0, 2000)

fig, ax = plt.subplots(figsize=(4, 3))

# Line styles for each DD level
styles = {'DD-1': '-', 'DD-2': '--', 'DD-3': '-.', 'DD-4': ':'}

for dd, params in DD_PARAMS.items():
    Sae = compute_spectrum(T_range, **params)
    ax.plot(T_range, Sae, styles[dd], color='k', linewidth=0.9, label=dd)

# Structure fundamental period
T1 = periods[0]
Sae_T1_DD2 = compute_spectrum(np.array([T1]), **DD_PARAMS['DD-2'])[0]
ax.axvline(x=T1, color='k', linestyle=':', linewidth=0.6, alpha=0.7)
ax.plot(T1, Sae_T1_DD2, 'ko', markersize=4, markerfacecolor='white', markeredgewidth=0.8)
ax.text(T1 + 0.02, Sae_T1_DD2 + 0.08, r'$T_1$', fontsize=8)

# Labels
ax.set_xlabel(r'$T$ (s)')
ax.set_ylabel(r'$S_{ae}$ (g)')
ax.set_xlim(0, 1.0)
ax.set_ylim(0, 1.8)
ax.legend(frameon=False, fontsize=7, loc='upper right')

plt.tight_layout()
plt.savefig(VIZ_DIR / 'tbdy2018_spectrum_dd_all.png', dpi=300, bbox_inches='tight')
plt.savefig(VIZ_DIR / 'tbdy2018_spectrum_dd_all.pdf', bbox_inches='tight')
plt.show()

print(f"Structure period: T1 = {T1:.4f} s")

## 8. Final Results Summary

### 8.1 Modal Analysis Results Table

In [ ]:
# ==============================================================================
# 8.1 FINAL SUMMARY TABLE
# ==============================================================================

# Determine spectrum region for each mode
modal_results['Region'] = np.select(
    [
        modal_results['Period (s)'] < SPEC['TA'],
        (modal_results['Period (s)'] >= SPEC['TA']) & (modal_results['Period (s)'] < SPEC['TB']),
        modal_results['Period (s)'] >= SPEC['TB']
    ],
    ['Ascending', 'Plateau', 'Descending'],
    default='Unknown'
)

print("=" * 80)
print("FINAL MODAL ANALYSIS RESULTS - DASK 2026 TWIN TOWERS V8")
print("=" * 80)
print(f"\nModel Statistics:")
print(f"  Total Nodes:     {len(pos_df)}")
print(f"  Total Elements:  {len(conn_df)}")
print(f"  Total Mass:      {TOTAL_MASS * 1000:.2f} kg")
print(f"  Height:          {X[:, 2].max():.1f} cm")
print(f"\n{'='*80}")

# Display final table
modal_results[['Mode', 'Period (s)', 'Frequency (Hz)', 'Sae (g)', 'Region']].style.format({
    'Period (s)': '{:.4f}',
    'Frequency (Hz)': '{:.2f}',
    'Sae (g)': '{:.3f}'
}).set_properties(**{'text-align': 'center'}).hide(axis='index')

### 8.2 Key Findings

1. **Fundamental Period**: $T_1 \approx 0.038$ s (Very Stiff)

2. **Spectrum Position**: All 8 modes fall in the **Ascending Region** ($T < T_A$)

3. **Design Advantage**:
   $$S_{ae}(T_1) < S_{DS} \implies \text{Reduced seismic demand}$$

4. **Conclusion**: The V8 design is optimized for earthquake resistance by maintaining high stiffness, which places it in the favorable ascending portion of the design spectrum.

In [ ]:
# ==============================================================================
# 8.3 EXPORT RESULTS
# ==============================================================================

# Save modal results to CSV
modal_results.to_csv(RESULTS_DIR / 'modal_periods_v2.csv', index=False)

# Save spectrum data
spectrum_df.to_csv(RESULTS_DIR / 'tbdy2018_spectrum.csv', index=False)

print("Results exported:")
print(f"  - {RESULTS_DIR / 'modal_periods_v2.csv'}")
print(f"  - {RESULTS_DIR / 'tbdy2018_spectrum.csv'}")
print(f"  - {VIZ_DIR / 'modal_analysis_tbdy2018.png'}")
print(f"  - {VIZ_DIR / 'modal_analysis_tbdy2018_zoom.png'}")

print("\n" + "=" * 80)
print("ANALYSIS COMPLETE")
print("=" * 80)